In [2]:
import pandas as pd
import re
import requests
from dateutil.relativedelta import relativedelta

data_raw = pd.read_csv("data_raw/participantes.csv", sep = ";").drop(axis=1, labels=["Unnamed: 0"])

Unindo colunas de nascimento e renomeando

In [63]:
cols = ['Data de nascimento', 'Idade', 'Data de Nascimento']
data = data_raw.assign(
    data_nascimento = lambda x: x['Data de nascimento'].fillna("") +
                                x['Data de Nascimento'].fillna("") +
                                x['Idade'].fillna("")
).drop(cols, axis = 1).drop("Ref.", axis = 1)
data = data.rename(columns = {"Participante": 'nome', "Ocupação": "ocupacao", "Resultado":"resultado"})

Corrigindo localidade de origem

In [191]:
data_local = data.assign(
    municipio = lambda x: x["Origem"].str.split(",").str[0],
    pais = lambda x: x["Origem"].str.split(",").str[1]
).drop(["Origem"], axis = 1)

data_local['pais'] = data_local['pais'].fillna("Exterior")

Corrigindo Resultado

In [140]:
num_participantes = data_local.groupby(["edicao"]).agg(
    n_participantes = ("edicao", "count")
)

participantes_edicao = data_local.join(num_participantes, on ='edicao').assign(
    resultado = lambda x: x['resultado'].str.split("em").str[0]
)

In [188]:
def arrumar_classificacao(row):
    # texto, n_participantes

    if any(i in row['resultado'] for i in ["Vencedor", "Vencedora"]):
        row['resultado'] = re.sub("Ven.*", "1", row['resultado'])
    elif any(i in row['resultado'] for i in ["eliminado", "eliminada"]):
        lugar = re.sub("ª eli.*|º eli.*", "", row['resultado'])
        lugar = row['n_participantes'] + 1 - int(lugar)
        row['resultado'] = str(lugar)
    elif "lugar" in row['resultado']: 
        row['resultado'] = row['resultado'][0]
    else:
        row['resultado'] = "99" 
    
    return row['resultado']

participantes_edicao['classificacao'] = participantes_edicao.apply(arrumar_classificacao, axis = 1)
participantes_edicao = participantes_edicao.drop(['resultado'], axis = 1)

- Gênero #Não rodar esse codigo

In [216]:
def genero(row):
    key = "usei o gender-api"
    primeiro_nome = row['nome'].split(" ")[0]
    url = f"https://gender-api.com/get?name={primeiro_nome}&key={key}"
    gender_api = requests.get(url)
    gender_json = eval(gender_api.text)
    gender = gender_json["gender"]

    return gender

participantes_edicao['genero'] = participantes_edicao.apply(genero, axis = 1)

In [225]:
participantes_edicao.to_csv("data_raw/participantes_genero.csv", sep = ";", index = False)

In [3]:
participantes_genero = pd.read_csv("data_raw/participantes_genero.csv", sep = ";")

- Idade quando o programa começou

In [4]:
from wikitable import wikitable

url = "https://pt.wikipedia.org/wiki/Big_Brother_Brasil"
temporadas_raw = wikitable(url)[2]

In [5]:
temporadas_raw.columns = temporadas_raw.columns.droplevel()

In [6]:
estreias = temporadas_raw[["Temporada.1", "Estreia da temporada"]].rename({"Temporada.1": "edicao", "Estreia da temporada": "estreia"}, axis = 1)

estreias['edicao'] = estreias['edicao'].str.split(" ").str[-1]
estreias['estreia'] = estreias['estreia'].str.replace(" de ", "/").str.replace("janeiro", "01").str.replace("maio", "05")

estreias['edicao'] = pd.to_numeric(estreias['edicao'])
participantes_estreia = participantes_genero.merge(estreias, on="edicao", how='left')

participantes_estreia[['data_nascimento', 'estreia']] = participantes_estreia[['data_nascimento', 'estreia']].apply(pd.to_datetime, infer_datetime_format=True)

participantes_estreia = participantes_estreia.assign(
    idade_participacao = lambda x: x['estreia'].dt.year - x['data_nascimento'].dt.year
).drop(labels=['estreia', 'n_participantes'], axis = 1)

In [7]:
participantes_estreia

,nome,ocupacao,edicao,data_nascimento,municipio,pais,classificacao,genero,idade_participacao
0,Kleber de Paula Pedra (Bambam),Dançarino,1,1978-02-14,Campinas,São Paulo,1,male,24.0
1,Vanessa Melanie Pascale Ekypenyong Walborn,Modelo,1,1974-11-28,Rio de Janeiro,Rio de Janeiro,2,female,28.0
2,André Batista de Carvalho,Professor de canto,1,1974-11-28,Rio de Janeiro,Rio de Janeiro,3,male,28.0
3,Antonio Sergio Tavares Campo,Cabeleireiro,1,1972-12-06,Angola,Exterior,4,male,30.0
4,Alessandra Begliomini,Empresária,1,1974-12-05,São Paulo,São Paulo,5,female,28.0
...,...,...,...,...,...,...,...,...,...
357,Bárbara Heck,Relações públicas e modelo,22,1992-07-13,Novo Hamburgo,Rio Grande do Sul,19,female,30.0
358,Vitória Nascimento Câmara (Maria),Atriz e cantora,22,2000-07-18,Rio de Janeiro,Rio de Janeiro,99,female,22.0
359,Naiara de Fátima Azevedo,Cantora,22,1989-10-30,Campo Mourão,Paraná,20,female,33.0
360,Rodrigo Abrão de Carvalho Mussi Ivo,Gerente comercial,22,1985-09-20,São José dos Campos,São Paulo,21,male,37.0


In [8]:
participantes_estreia = participantes_estreia[['nome', 'genero', "data_nascimento", "idade_participacao", "ocupacao", "municipio", "pais", "classificacao"]]
participantes_estreia


,nome,genero,data_nascimento,idade_participacao,ocupacao,municipio,pais,classificacao
0,Kleber de Paula Pedra (Bambam),male,1978-02-14,24.0,Dançarino,Campinas,São Paulo,1
1,Vanessa Melanie Pascale Ekypenyong Walborn,female,1974-11-28,28.0,Modelo,Rio de Janeiro,Rio de Janeiro,2
2,André Batista de Carvalho,male,1974-11-28,28.0,Professor de canto,Rio de Janeiro,Rio de Janeiro,3
3,Antonio Sergio Tavares Campo,male,1972-12-06,30.0,Cabeleireiro,Angola,Exterior,4
4,Alessandra Begliomini,female,1974-12-05,28.0,Empresária,São Paulo,São Paulo,5
...,...,...,...,...,...,...,...,...
357,Bárbara Heck,female,1992-07-13,30.0,Relações públicas e modelo,Novo Hamburgo,Rio Grande do Sul,19
358,Vitória Nascimento Câmara (Maria),female,2000-07-18,22.0,Atriz e cantora,Rio de Janeiro,Rio de Janeiro,99
359,Naiara de Fátima Azevedo,female,1989-10-30,33.0,Cantora,Campo Mourão,Paraná,20
360,Rodrigo Abrão de Carvalho Mussi Ivo,male,1985-09-20,37.0,Gerente comercial,São José dos Campos,São Paulo,21


In [10]:
participantes_estreia.to_csv("data/dados.csv", sep =';', index = False)